In [ ]:
if __name__ == '__main__':
    import import_ipynb, os, sys
    sys.path.append(os.environ['PROJECT_ROOT'])

In [ ]:
from fastapi import FastAPI, APIRouter, Header, HTTPException, Request, Depends, Path, Response, Cookie, File, UploadFile
from fastapi.responses import JSONResponse, HTMLResponse, PlainTextResponse
from fastapi.exceptions import RequestValidationError
from fastapi.openapi.utils import get_openapi
from starlette.middleware.cors import CORSMiddleware
from starlette.staticfiles import StaticFiles
from fastapi.security import OAuth2PasswordBearer, OAuth2PasswordRequestForm
from starlette.status import HTTP_422_UNPROCESSABLE_ENTITY
from fastapi.templating import Jinja2Templates
from fastapi.security import APIKeyHeader
from pydantic import constr, Field
from typing import Optional
import sys, traceback, httpx, json, pymongo, bson

In [ ]:
from app.Validations.Validation import MongoModel, PyObjectId
import app.Exceptions.HttpException as HEP
from app.kernel import env
from app.Enums.ErrorCodeEnum import ErrorCodeEnum
from app.Validations.Validation import ResponseBase, ValidationRules

In [ ]:
class Helper:

    @staticmethod
    def exception_handler(e, request):
        request_dict = request.__dict__
        if not hasattr(e, 'code') or not hasattr(e, 'error'):
            e = HEP.InternalServerError500Exception()
            log_data = e.__dict__
            log_data['error'] = traceback.format_exc() if not log_data['error'] else log_data['error']
            log_data['request'] = {
                'client': {'host': request.client.host, 'port': request.client.port},
                'server': {'host': request_dict['scope']['server'][0], 'port': request_dict['scope']['server'][1]},
                'headers': dict(request.headers),
                'url': str(request.url),
                'scope': request_dict['scope']['root_path'],
                'method': request.method,
                'scheme': request_dict['scope']['scheme']
            }
            log_data['created_at'] = DateService.now()
        except_data = ResponseBase(data=e.data, msg=e.msg, status_code=e.code, result=False).dict()
        return JSONResponse(status_code=e.code, content=except_data)

    apiKey = OAuth2PasswordBearer(tokenUrl="token")

    @staticmethod
    async def csrf_authorize(token: str = Depends(apiKey)):
        Auth().store(token)

In [ ]:
class API:
    auth = APIRouter(prefix='/api/auth')
    router = APIRouter(dependencies=[Depends(Helper.csrf_authorize)], prefix='/science')

In [ ]:
app = FastAPI(docs_url=None)

app.include_router(API.router)
@app.middleware("http")
async def middleware_manager(request: Request, call_next):

    print('api middleware start')
    try:
        response = await call_next(request)
    except bson.errors.InvalidId:
        e = ErrorCodeEnum.INVALID_OBJECT_ID
        response = Helper.exception_handler(e, request)
    except Exception as e:
        response = Helper.exception_handler(e, request)
        print(e)
    print('api middleware end')
    return response

app.add_middleware(
    CORSMiddleware,
    allow_origins=[env['SERVER_NAME'], '*.{}'.format(env['SERVER_NAME'])],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=['Content-Type', 'X-Requested-With', 'Accept', 'X-CSRF-TOKEN', 'Authorization', 'X-Client-Token']
)


def custom_openapi():
    if app.openapi_schema:
        return app.openapi_schema
    openapi_schema = get_openapi(
        title="Data Science",
        version="1.0.0",
        description="",
        routes=app.routes,
    )
    app.openapi_schema = openapi_schema
    return app.openapi_schema

app.openapi = custom_openapi